In [14]:
import pandas as pd
import talib as TA
import pandas_ta as ta
import matplotlib.pyplot as plt
from backtesting import Strategy, Backtest

In [15]:
'''
This strategy takes into account RSI for entry signals and a exponential moving average for trend detection.
is intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.
The backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.
'''

'\nThis strategy takes into account RSI for entry signals and a exponential moving average for trend detection.\nis intended to be used with a 15m timeframe, but can be used with any timeframe, just test it out.\nThe backtest is done with backtesting.py and is intended to throw the results from it, the results are going to be optimized.\n'

READ & CLEAN THE DATA SET

In [16]:
df = pd.read_csv('data_sets/EURUSD_Candlestick_15_M_BID_09.04.2004-04.04.2022.csv')
df = df[df['Volume']!=0]
df.drop(columns=['Unnamed: 0'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Local time,Open,High,Low,Close,Volume
0,09.04.2004 00:00:00.000 GMT-0500,1.20722,1.20820,1.20689,1.20794,7953.20
1,09.04.2004 00:15:00.000 GMT-0500,1.20796,1.20855,1.20752,1.20824,7377.60
2,09.04.2004 00:30:00.000 GMT-0500,1.20817,1.20838,1.20767,1.20790,8624.70
3,09.04.2004 00:45:00.000 GMT-0500,1.20807,1.20834,1.20719,1.20820,6816.70
4,09.04.2004 01:00:00.000 GMT-0500,1.20780,1.20941,1.20773,1.20888,8339.30
...,...,...,...,...,...,...
449188,04.04.2022 17:45:00.000 GMT-0500,1.09742,1.09742,1.09717,1.09724,154.88
449189,04.04.2022 18:00:00.000 GMT-0500,1.09727,1.09735,1.09716,1.09722,148.93
449190,04.04.2022 18:15:00.000 GMT-0500,1.09722,1.09730,1.09710,1.09723,199.63
449191,04.04.2022 18:30:00.000 GMT-0500,1.09723,1.09761,1.09722,1.09745,184.76


SLICING THE DATASET

In [17]:
#60% of the dataset
df60 = df[0:270000]  
#40% of the dataset
df40 = df[270000:449192]  
df60.dropna()
df40.dropna()

,Local time,Open,High,Low,Close,Volume
270000,27.01.2015 04:30:00.000 GMT-0500,1.13079,1.13284,1.13065,1.13231,2730.35
270001,27.01.2015 04:45:00.000 GMT-0500,1.13230,1.13289,1.13168,1.13179,3129.66
270002,27.01.2015 05:00:00.000 GMT-0500,1.13179,1.13182,1.13090,1.13096,2457.01
270003,27.01.2015 05:15:00.000 GMT-0500,1.13096,1.13124,1.12828,1.12857,3637.84
270004,27.01.2015 05:30:00.000 GMT-0500,1.12857,1.13041,1.12821,1.12907,3225.63
...,...,...,...,...,...,...
449187,04.04.2022 17:30:00.000 GMT-0500,1.09740,1.09748,1.09729,1.09739,145.72
449188,04.04.2022 17:45:00.000 GMT-0500,1.09742,1.09742,1.09717,1.09724,154.88
449189,04.04.2022 18:00:00.000 GMT-0500,1.09727,1.09735,1.09716,1.09722,148.93
449190,04.04.2022 18:15:00.000 GMT-0500,1.09722,1.09730,1.09710,1.09723,199.63


BACKTEST ATR RELATED SL&TP

In [ ]:
class MyStrat(Strategy):
    ema_window = 200
    rsi_window = 2
    atr_window = 14
    riskpct = 0.4
    
    def init(self):
        super().init()
        self.ema = self.I(TA.EMA, self.data.Close, self.ema_window)
        self.rsi = self.I(TA.RSI, self.data.Close, self.rsi_window)   
        self.atr = self.I(TA.ATR, self.data.Close, self.data.High, self.data.Low, self.atr_window) 

    def next(self):
        super().next()
        slatr = 1.2*self.atr[-1]
        TPSLRatio = 2
        price = self.data.Close[-1] 
        
        if self.position:
            self.position.close()
            
        if self.data.Close >= self.ema and self.rsi <= 10:
            sl1 = price - slatr
            tp1 = price + slatr*TPSLRatio    # 1e-4 is 1 pip
             
            
            self.buy(sl=sl1, tp=tp1, size= self.riskpct)
            
        if self.data.Close <= self.ema and self.rsi >= 90:
            sl2 = price + slatr
            tp2 = price - slatr*TPSLRatio    # 1e-4 is 1 pip
            
                                    
            self.sell(sl=sl2, tp=tp2, size= self.riskpct)

bt = Backtest(df40, MyStrat, cash=100, margin=1/50, commission = 0.00, trade_on_close=False)
stat = bt.run()
print(stat)
trades_size = stat['_trades']['Size'] 
print(trades_size) # shows the size of the trades
#plt.bar(trades_size, height=trades_size.count())
#plot = bt.plot(plot_volume=False)
#print(plot)

OPTIMIZATION

In [ ]:
# Since Optimization takes some time to analize the data, I already have done it and the best parameters are: EMA=200, RSI=2

# stats = bt.optimize(
#     ema_window = [50, 200],
#     rsi_window = range(2, 4),
#     maximize = 'Equity Final [$]',
# )
# print(stats)
# print(stats._strategy)
# bt.plot(plot_volume=False, plot_pl=False)

CALCULATION OF POSITION SIZE BY RISK% AND SL SIZE

In [ ]:
# c, h, o, l = self.data.Close[-1], self.data.High[-1], self.data.Open[-1], self.data.Low[-1]
# riskpct = 0.002
# contract_size = 100000
# size2 = (self.equity*riskpct)/(h - c)*contract_size   
# size1 = (self.equity*riskpct)/(o - l)*contract_size

CALCULATION OF SL ON LOW AND HIGH OF SIGNAL CANDLE 

In [ ]:
# c, h, o, l = self.data.Close[-1], self.data.High[-1], self.data.Open[-1], self.data.Low[-1]
# self.sell(limit=c, sl=h+4e-4, tp=c - 2*(c-1), size=riskpct)
# self.buy(limit=c, sl=l-4e-4, tp=c + 2*(c-1), size=riskpct)